[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/upm-classes/image_mining_2021_2022/blob/main/classification_image_mining.ipynb)

# Dataset

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

In [ ]:
ds_name = 'rock_paper_scissors'

image_train, label_train = tfds.as_numpy(tfds.load(
    ds_name,
    split='train',
    batch_size=-1,
    as_supervised=True,
))
image_test, label_test = tfds.as_numpy(tfds.load(
    ds_name,
    split='test',
    batch_size=-1,
    as_supervised=True, 
))

In [ ]:
print(f'Number images in train set: {image_train.shape[0]}')
print(f'Number images in test set: {image_test.shape[0]}')
print(f'Labels in train set: {np.unique(label_train)}')
print(f'Labels in test set: {np.unique(label_test)}')

### Visualization

In [ ]:
import matplotlib.pyplot as plt

num_samples = 10

rand_samples = np.random.choice(np.arange(image_train.shape[0]), num_samples)

fig = plt.figure(figsize=(20, 8))
for i in range(num_samples):
    
    im = image_train[rand_samples[i]]
    label = label_train[rand_samples[i]]
    ax = fig.add_subplot(2, 5, i+1)
    ax.set_title(f"Label: {label}")
    ax.axis('off')
    ax.imshow(im)
plt.show()

In [ ]:
num_samples = 10

rand_samples = np.random.choice(np.arange(image_test.shape[0]), num_samples)

fig = plt.figure(figsize=(20, 8))
for i in range(num_samples):
    
    im = image_test[rand_samples[i]]
    label = label_test[rand_samples[i]]
    ax = fig.add_subplot(2, 5, i+1)
    ax.set_title(f"Label: {label}")
    ax.axis('off')
    ax.imshow(im)
plt.show()

In [ ]:
print('Information about classes in train set:')

classes = np.unique(label_train)
description = np.array([(c, np.sum(label_train == c)) for c in classes])

for desc in description:
    print('Class: {}, number of samples: {}'.format(desc[0], desc[1]))

plt.bar(description[:,0], description[:,1].astype(int))
plt.show()

In [ ]:
print('Information about classes in test set:')

classes = np.unique(label_test)
description = np.array([(c, np.sum(label_test == c)) for c in classes])

for desc in description:
    print('Class: {}, number of samples: {}'.format(desc[0], desc[1]))

plt.bar(description[:,0], description[:,1].astype(int))
plt.show()

## Classification with histogram as a feature

In [ ]:
import cv2 
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import ConfusionMatrixDisplay

from joblib import dump, load
import os

In [ ]:
# Test set

x_train = image_train.astype(np.uint8)

x_train = np.array([np.concatenate([cv2.calcHist(x_train[i],[0],None,[256],[0,256]), 
                cv2.calcHist(x_train[i],[1],None,[256],[0,256]),
                cv2.calcHist(x_train[i],[2],None,[256],[0,256])]) 
for i in range(x_train.shape[0])])
x_train = np.squeeze(x_train)

max_value = np.max(x_train)
x_train = x_train / max_value

# Test set
x_test = image_test.astype(np.uint8)

x_test = np.array([np.concatenate([cv2.calcHist(x_test[i],[0],None,[256],[0,256]), 
                cv2.calcHist(x_test[i],[1],None,[256],[0,256]),
                cv2.calcHist(x_test[i],[2],None,[256],[0,256])]) 
for i in range(x_test.shape[0])])
x_test = np.squeeze(x_test)

x_test = x_test / max_value

print(f'Size of the train set: {x_train.shape}')
print(f'Size of the test set: {x_test.shape}')

plt.plot(x_train[0])
plt.show()

plt.plot(x_test[0])
plt.show()

In [ ]:
model_name = 'svm_model_1_pixels.joblib'

model = svm.SVC() # You can set the parameters

if not os.path.exists(model_name):
  model.fit(x_train, label_train) # Fitting the model
  dump(model, model_name) # saving model
else:
  model = load(model_name) # loading model

In [ ]:
predicted_train = model.predict(x_train)

print("Classification report for classifier")
print(f"{classification_report(label_train, predicted_train)}")

disp = ConfusionMatrixDisplay.from_predictions(label_train, predicted_train)
disp.figure_.suptitle("Confusion Matrix")
plt.show()

In [ ]:
predicted_test = model.predict(x_test)

In [ ]:
# Visualize some predictions

num_samples = 10

rand_samples = np.random.choice(np.arange(x_test.shape[0]), num_samples)

fig = plt.figure(figsize=(20, 8))
for i in range(num_samples):
    im = image_test[rand_samples[i]]
    label = label_test[rand_samples[i]]
    predicted_label = predicted_test[rand_samples[i]]
    ax = fig.add_subplot(2, 5, i+1)
    ax.set_title(f"Label: {label} -- Prediction: {predicted_label}")
    ax.axis('off')
    ax.imshow(im)
plt.show()

In [ ]:
print("Classification report for classifier")
print(f"{classification_report(label_test, predicted_test)}")

disp = ConfusionMatrixDisplay.from_predictions(label_test, predicted_test)
disp.figure_.suptitle("Confusion Matrix")
plt.show()

## Classification using deep features

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input

from tqdm import tqdm
import os

![alt](https://1.bp.blogspot.com/-nJlr9zzycPs/XOfawrPqhUI/AAAAAAAAADQ/t2WtEQDrboMPlHgvDEJsk48rkDzC_KGaACLcBGAs/s1600/resnet50.JPG "Resnet-50")

In [ ]:
model = ResNet50(weights="imagenet", include_top=False)
model.summary()

In [ ]:
x_train = []

for i in tqdm(range(image_train.shape[0])):
  im = image_train[i].astype(np.uint8)
  im = cv2.resize(im, (224, 224))
  im = preprocess_input(im)
  features = model.predict(np.expand_dims(im, axis=0)) # size 1 x 7 x 7 x 2048
  features = features.reshape((features.shape[1] * features.shape[2] * 
                               features.shape[3]))

  x_train.append(features)
x_train = np.array(x_train)

x_test = []

for i in tqdm(range(image_test.shape[0])):
  im = image_test[i].astype(np.uint8)
  im = cv2.resize(im, (224, 224))
  im = preprocess_input(im)
  features = model.predict(np.expand_dims(im, axis=0)) # size 1 x 7 x 7 x 2048
  features = features.reshape((features.shape[1] * features.shape[2] * 
                               features.shape[3])) #100352

  x_test.append(features)
x_test = np.array(x_test)

In [ ]:
im = image_test[i].astype(np.uint8)
im = cv2.resize(im, (224, 224))
im = preprocess_input(im)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10,5))
ax1.imshow(image_test[i].astype(np.uint8))
ax2.imshow((im + np.abs(np.min(im)))/np.max(im  + np.abs(np.min(im)))) # range [0,1]
plt.show()

features = model.predict(np.expand_dims(im, axis=0))
num_samples = 10

rand_samples = np.random.choice(np.arange(features.shape[3]), num_samples)

fig = plt.figure(figsize=(20, 8))
for i in range(num_samples):
    img = features[0, :, :, rand_samples[i]]
    ax = fig.add_subplot(2, 5, i+1)
    ax.set_title(f'Feature {rand_samples[i]}')
    ax.axis('off')
    ax.imshow(img)


plt.show()

In [ ]:
model_name = 'svm_model_2_pixels.joblib'

model = svm.SVC() # You can set the parameters

if not os.path.exists(model_name):
  model.fit(x_train, label_train) # Fitting the model
  dump(model, model_name) # saving model
else:
  model = load(model_name) # loading model

In [ ]:
predicted_train = model.predict(x_train)

print("Classification report for classifier")
print(f"{classification_report(label_train, predicted_train)}")

disp = ConfusionMatrixDisplay.from_predictions(label_train, predicted_train)
disp.figure_.suptitle("Confusion Matrix")
plt.show()

In [ ]:
predicted = model.predict(x_test)

In [ ]:
num_samples = 10

rand_samples = np.random.choice(np.arange(x_test.shape[0]), num_samples)

fig = plt.figure(figsize=(20, 8))
for i in range(num_samples):
    
    im = image_test[rand_samples[i]]
    label = label_test[rand_samples[i]]
    predicted_label = predicted[rand_samples[i]]
    ax = fig.add_subplot(2, 5, i+1)
    ax.set_title(f"Label: {label} -- Prediction: {predicted_label}")
    ax.axis('off')
    ax.imshow(im)
plt.show()

In [ ]:
print("Classification report for classifier")
print(f"{classification_report(label_test, predicted)}")

disp = ConfusionMatrixDisplay.from_predictions(label_test, predicted)
disp.figure_.suptitle("Confusion Matrix")
plt.show()

## Classification using Deep features with PCA

PCA is a dimensionality reduction method that identifies important relationships in data, transforms existing data based on these relationships, and then quantifies the importance of these relationships so that we can keep the most important ones and drop the others (correlated data). 

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()
pca.fit(x_train)

cumsum = np.cumsum(pca.explained_variance_ratio_)
selected_components = np.where(cumsum >= 0.95)[0][0] + 1

print(f'Explained variance: {cumsum[selected_components]} by {selected_components} components')

In [ ]:
pca = PCA(n_components=selected_components)
pca.fit(x_train)

print(f'Explained variance: {np.sum(pca.explained_variance_ratio_)} by {selected_components} components')

In [ ]:
x_train_pca = pca.transform(x_train)
x_test_pca = pca.transform(x_test)

In [ ]:
model_name = 'svm_model_3_pixels.joblib'

model = svm.SVC() # You can set the parameters

if not os.path.exists(model_name):
  model.fit(x_train_pca, label_train) # Fitting the model
  dump(model, model_name) # saving model
else:
  model = load(model_name) # loading model

In [ ]:
predicted_train = model.predict(x_train_pca)

print("Classification report for classifier")
print(f"{classification_report(label_train, predicted_train)}")

disp = ConfusionMatrixDisplay.from_predictions(label_train, predicted_train)
disp.figure_.suptitle("Confusion Matrix")
plt.show()

In [ ]:
predicted = model.predict(x_test_pca)

In [ ]:
num_samples = 10

rand_samples = np.random.choice(np.arange(x_test.shape[0]), num_samples)

fig = plt.figure(figsize=(20, 8))
for i in range(num_samples):
    
    im = image_test[rand_samples[i]]
    label = label_test[rand_samples[i]]
    predicted_label = predicted[rand_samples[i]]
    ax = fig.add_subplot(2, 5, i+1)
    ax.set_title(f"Label: {label} -- Prediction: {predicted_label}")
    ax.axis('off')
    ax.imshow(im)
plt.show()

In [ ]:
print("Classification report for classifier")
print(f"{classification_report(label_test, predicted)}")

disp = ConfusionMatrixDisplay.from_predictions(label_test, predicted)
disp.figure_.suptitle("Confusion Matrix")
plt.show()

# Classifiying by training a pretrained neural network

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model

from tensorflow.keras.layers import Input, Flatten, Dense 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from skimage.transform import resize

from tqdm import tqdm
import os
from pickle import dump

### Dataset

In [ ]:
ds_name = 'rock_paper_scissors'

image_train, label_train = tfds.as_numpy(tfds.load(
    ds_name,
    split='train',
    batch_size=-1,
    as_supervised=True,
))
image_test, label_test = tfds.as_numpy(tfds.load(
    ds_name,
    split='test',
    batch_size=-1,
    as_supervised=True, 
))

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(np.arange(image_train.shape[0]),
                                                  label_train, 
                                                  test_size=0.20, 
                                                  random_state=23)

x_train = image_train[x_train, ...]
x_val = image_train[x_val, ...]
x_test = image_test
y_test = label_test

print('Training set shape: {}'.format(x_train.shape))
print('Validation set shape: {}'.format(x_val.shape))
print('Test set shape: {}'.format(x_test.shape))

In [ ]:
x_train = x_train[:280]
y_train = y_train[:280]

x_val = x_val[:280]
y_val = y_val[:280]

### Preprocessing

In [ ]:
# Resizing images to (224, 224) and rescale to [0,1] range
x_train = np.array([resize(x_train[i, :, :, :], (224, 224)) for i in range(x_train.shape[0])])
x_val = np.array([resize(x_val[i, :, :, :], (224, 224)) for i in range(x_val.shape[0])])
x_test = np.array([resize(x_test[i, :, :, :], (224, 224)) for i in range(x_test.shape[0])])

In [ ]:
# One hot encoder labels
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(y_train.reshape(-1, 1))

y_train = enc.transform(y_train.reshape(-1, 1)).toarray()
y_val = enc.transform(y_val.reshape(-1, 1)).toarray()

### Dataset

In [ ]:
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

ds_train = ds_train.batch(56) # batch size
ds_val = ds_val.batch(56)

### Model

In [ ]:
n_classes = 3
input_shape = (224, 224, 3)

base_model = ResNet50(weights="imagenet", 
                      input_shape=input_shape, 
                      include_top=False)

inputs = Input(shape=input_shape)
x = base_model(inputs, training=True)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = Flatten()(x)
x = Dense(1024, activation='relu')(x)
outputs = Dense(n_classes, activation='softmax')(x)
model = Model(inputs, outputs)

model.summary()

In [ ]:
# we need to recompile our model
model.compile(optimizer=Adam(1e-5),  # 1e-5 learning rate
              loss=CategoricalCrossentropy(),
              metrics=[CategoricalAccuracy()])

### Training

In [ ]:
num_epochs = 5
callbacks = [
    EarlyStopping(patience=2),
    ModelCheckpoint(filepath='model.{epoch:04d}-{val_loss:.8f}.h5'),
    CSVLogger('training.log'),
]

model.fit(ds_train, validation_data=ds_val, epochs=num_epochs, 
          callbacks=callbacks)